In [6]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from qiskit_machine_learning.kernels import FidelityQuantumKernel
#Añadimos ComputeUncompute
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler   # IMPORTANTE: reemplaza Aer para kernels

import numpy as np

# 1) Dataset
X, y = datasets.make_moons(n_samples=500, noise=0.2, random_state=42)

# Normalizar
X = StandardScaler().fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# 2) Modelos con kernels clásicos
kernels = {
    "Lineal": SVC(kernel="linear"),
    "RBF": SVC(kernel="rbf", gamma='scale'),
    "Polynomial": SVC(kernel="poly", degree=3)
}

print("Entrenando modelos clásicos...\n")

for name, model in kernels.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"{name}: {acc:.3f}")


# 3) Quantum Kernel moderno
print("\nEntrenando modelo con Quantum Kernel...\n")

# FeatureMap cuántico (mismo usado en el artículo)
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)

# Sampler moderno (reemplaza por completo a Aer)
sampler = Sampler()

# 2. Creamos instancia de Fidelity
fidelity_instance = ComputeUncompute(sampler=sampler)

quantum_kernel = FidelityQuantumKernel(
    feature_map=feature_map,
    fidelity = fidelity_instance
)

# MATRIZ KERNEL (train y test)
K_train = quantum_kernel.evaluate(X_train)
K_test = quantum_kernel.evaluate(X_test, X_train)

# SVM con kernel precomputado
svc_q = SVC(kernel='precomputed')

svc_q.fit(K_train, y_train)
preds_q = svc_q.predict(K_test)
acc_q = accuracy_score(y_test, preds_q)

print(f"Quantum Kernel (ZZFeatureMap): {acc_q:.3f}")


Entrenando modelos clásicos...

Lineal: 0.867
RBF: 0.973
Polynomial: 0.887

Entrenando modelo con Quantum Kernel...



/tmp/ipykernel_2769/1226985081.py:48: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/tmp/ipykernel_2769/1226985081.py:51: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  fidelity_instance = ComputeUncompute(sampler=sampler)


Quantum Kernel (ZZFeatureMap): 0.627
